# Objectives:
1. Determine the proper SUMLEV for analyzing city population growth.
2. Preprocess City Names and Combine All City Parts.
3. Calculate YoY Growth.
4. Transform into OLAP Form.

In [61]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:,.6f}'.format
import numpy as np

In [62]:
datafile = r'./Data/USA_Population.csv'
data = pd.read_csv(datafile, encoding='latin-1')

In [63]:
data.head()

,SUMLEV,STATE,COUNTY,PLACE,COUSUB,CONCIT,PRIMGEO_FLAG,FUNCSTAT,NAME,STNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019
0,40,1,0,0,0,0,0,A,Alabama,Alabama,4779736,4780125,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185
1,162,1,0,124,0,0,0,A,Abbeville city,Alabama,2688,2705,2699,2694,2643,2628,2608,2600,2584,2575,2571,2560
2,162,1,0,460,0,0,0,A,Adamsville city,Alabama,4522,4506,4500,4493,4471,4449,4420,4390,4356,4327,4308,4281
3,162,1,0,484,0,0,0,A,Addison town,Alabama,758,754,751,750,743,742,739,734,731,726,723,718
4,162,1,0,676,0,0,0,A,Akron town,Alabama,356,356,355,347,347,343,338,339,333,332,331,328


In [64]:
data.shape

(81434, 22)

In [65]:
data.iloc[:,7:].describe()

,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019
count,"81,434.000000","81,434.000000","81,434.000000","81,434.000000","81,434.000000","81,434.000000","81,434.000000","81,434.000000","81,434.000000","81,434.000000","81,434.000000"
mean,"15,847.126483","15,873.921237","15,980.608579","16,089.079377","16,192.618710","16,300.638591","16,407.461687","16,512.188066","16,605.111661","16,681.620294","16,750.781504"
std,"242,179.623178","242,640.987882","244,697.624913","246,809.606434","248,805.399550","250,991.841403","253,196.517623","255,195.170875","256,854.686975","258,101.712547","259,206.398671"
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,342.000000,342.000000,341.000000,340.000000,339.000000,339.000000,337.000000,337.000000,336.000000,336.000000,335.000000
50%,"1,212.000000","1,213.500000","1,214.500000","1,214.000000","1,212.000000","1,211.000000","1,210.000000","1,207.000000","1,205.500000","1,204.000000","1,205.000000"
75%,"5,079.250000","5,083.750000","5,093.000000","5,100.000000","5,097.000000","5,109.000000","5,115.750000","5,120.000000","5,127.000000","5,143.000000","5,143.000000"
max,"37,254,519.000000","37,319,502.000000","37,638,369.000000","37,948,800.000000","38,260,787.000000","38,596,972.000000","38,918,045.000000","39,167,117.000000","39,358,497.000000","39,461,588.000000","39,512,223.000000"


---
### <b>Objective #1</b>: Determine Appropriate Sum Level.
<li>Which Sum Levels contain the entire USA population at the lowest granularity?</li>

In [66]:
data.groupby('SUMLEV').agg(sum)

,STATE,COUNTY,PLACE,COUSUB,CONCIT,PRIMGEO_FLAG,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019
SUMLEV,,,,,,,,,,,,,,,,,
40,1477,0,0,0,0,0,308758105,309321666,311556874,313830990,315993715,318301008,320635163,322941311,324985539,326687501,328239523
50,95140,325426,0,0,0,61,308758105,309321666,311556874,313830990,315993715,318301008,320635163,322941311,324985539,326687501,328239523
61,705089,1524404,0,887546895,0,12747,100472681,100545025,100797883,101006511,101237025,101418884,101474492,101546096,101679163,101778410,101821142
71,402222,1156245,800883139,589669329,0,13839,68533052,68580336,68749723,68905956,69073969,69191423,69225666,69262225,69318224,69346395,69347180
157,706243,2218467,1156659614,0,0,14076,304789222,305342915,307542901,309783922,311916412,314195756,316504956,318786813,320804794,322483165,324013328
162,578567,0,807007047,0,0,0,193978195,194350542,195907558,197527235,199045408,200613706,202204513,203698127,204949444,205950315,206884849
170,171,0,0,0,230943,0,2602769,2607376,2626533,2656243,2684734,2702209,2722641,2738820,2748980,2758890,2768798
172,2505,0,5227409,0,5025788,0,2602769,2607376,2626533,2656243,2684734,2702209,2722641,2738820,2748980,2758890,2768798


In [67]:
# Add feature for mean-population across the decade.

pop_estimates = ['POPESTIMATE2010','POPESTIMATE2011', 'POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015','POPESTIMATE2016','POPESTIMATE2017','POPESTIMATE2018','POPESTIMATE2019' ]

# Add vectors together to the number bank and divide by count of vectors.
number_bank = 0
for estimate_vec in pop_estimates:
    number_bank += data[estimate_vec]
decade_avg = number_bank/len(pop_estimates)

data['POPESTIMATE_MEAN'] = decade_avg.round(0).astype(int)
data.head()

,SUMLEV,STATE,COUNTY,PLACE,COUSUB,CONCIT,PRIMGEO_FLAG,FUNCSTAT,NAME,STNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,POPESTIMATE_MEAN
0,40,1,0,0,0,0,0,A,Alabama,Alabama,4779736,4780125,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185,4845320
1,162,1,0,124,0,0,0,A,Abbeville city,Alabama,2688,2705,2699,2694,2643,2628,2608,2600,2584,2575,2571,2560,2616
2,162,1,0,460,0,0,0,A,Adamsville city,Alabama,4522,4506,4500,4493,4471,4449,4420,4390,4356,4327,4308,4281,4400
3,162,1,0,484,0,0,0,A,Addison town,Alabama,758,754,751,750,743,742,739,734,731,726,723,718,736
4,162,1,0,676,0,0,0,A,Akron town,Alabama,356,356,355,347,347,343,338,339,333,332,331,328,339


<b>Analyze:</b> Sum Level 157 (incorportated places) vs. Sum Level 40 (state).

In [68]:
# Make a dataframe for state population in sumlevel 157 and 40 and then get the difference.
df_157 = data[data['SUMLEV']==157].groupby(['STNAME', 'SUMLEV']).agg(sum)[['POPESTIMATE_MEAN']].rename(columns={"POPESTIMATE_MEAN": 'Population_157'})

df_40 = data[data['SUMLEV']==40][['STNAME','POPESTIMATE_MEAN']].rename(columns={"POPESTIMATE_MEAN": 'Population_40'})

df_compare = pd.merge(df_157, df_40, how='left', on=['STNAME'])

df_compare['PopDiff'] = df_compare['Population_40'] - df_compare['Population_157']

df_compare.head(10)

,STNAME,Population_157,Population_40,PopDiff
0,Alabama,4845321,4845320,-1
1,Alaska,727340,732517,5177
2,Arizona,6804948,6804947,-1
3,Arkansas,2973842,2973844,2
4,California,38586292,38618190,31898
5,Colorado,5403200,5403199,-1
6,Connecticut,3430962,3582668,151706
7,Delaware,936445,936446,1
8,District of Columbia,663628,663628,0
9,Florida,20109632,20109631,-1


In [69]:
sumlev157_avgtotalpop = df_compare.Population_157.sum()
sumlev40_avgtotalpop =  df_compare.Population_40.sum()
dif_avgtotalpop = df_compare.Population_40.sum() - df_compare.Population_157.sum()
print('Total Population:\nSUMLEV 157: ' +"{:,}".format(sumlev157_avgtotalpop) +'\nSUMLEV 40: ' + "{:,}".format(sumlev40_avgtotalpop) + '\nDifference: ' + "{:,}".format(dif_avgtotalpop))

Total Population:
SUMLEV 157: 315,137,436
SUMLEV 40: 319,249,332
Difference: 4,111,896


In [70]:
states_w_missing = df_compare[df_compare['PopDiff'] != 0 ]

no_missing = df_compare[df_compare['PopDiff'] > 10000]['PopDiff'].count()
mean = df_compare[df_compare['PopDiff'] != 0 ]['PopDiff'].mean()
median = df_compare[df_compare['PopDiff'] != 0 ]['PopDiff'].median()
mini = df_compare[df_compare['PopDiff'] != 0 ]['PopDiff'].min()
maxi = df_compare[df_compare['PopDiff'] != 0 ]['PopDiff'].max()

print(str(no_missing) + ' states in Sum Level 157 averaged missing populaton (x > 10,000 People) from Sum Level 40.' + '\nAverage: ' +str(round(mean,1))+' \nMedian: ' + str(median) )

15 states in Sum Level 157 averaged missing populaton (x > 10,000 People) from Sum Level 40.
Average: 89389.0 
Median: 11.0


In [71]:
df_popdiff = states_w_missing[['STNAME', 'PopDiff']].sort_values('PopDiff', ascending=False )
df_popdiff.head(15)

,STNAME,PopDiff
46,Virginia,1523254
20,Maryland,1131119
11,Hawaii,427216
19,Maine,190219
39,Rhode Island,175349
18,Louisiana,170510
6,Connecticut,151706
28,Nevada,108551
29,New Hampshire,47884
4,California,31898


---
### <b>Objective #2</b>: Pre-Process City-Names
1. Remove '(pt.)' from City Parts
2. Remove the excess trailing word: 'city', 'town', 'village'
3. Aggregate All City Parts Into 1 Record

In [72]:
# Filter to Sum Level 157
data157 = data.copy()[data['SUMLEV'] == 157]

In [73]:
# Clean Up Columns
drop_columns = ['SUMLEV', 'STATE', 'COUNTY', 'PLACE', 'COUSUB', 'CONCIT', 'PRIMGEO_FLAG', 'FUNCSTAT', 'CENSUS2010POP', 'ESTIMATESBASE2010']
data157.drop(drop_columns, axis=1, inplace= True)
data157.head()

,NAME,STNAME,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,POPESTIMATE_MEAN
464,Autaugaville town,Alabama,888,890,883,869,869,866,869,870,870,871,874
465,Billingsley town,Alabama,148,148,147,144,144,143,143,144,142,142,144
466,Millbrook city (pt.),Alabama,171,172,172,169,170,171,171,172,171,173,171
467,Prattville city (pt.),Alabama,32362,32780,32669,32846,33008,33058,33369,33523,33690,33986,33129
468,Balance of Autauga County,Alabama,21204,21237,21083,20699,20702,20626,20691,20681,20660,20697,20828


In [74]:
# How Many Words in each City Name?
data157['NAME'] = data157['NAME'].str.split()
data157['NameLength'] = data157['NAME'].apply(lambda x: len(x))
data157['NameLength'].value_counts()

2    14031
3     5854
4     3604
5      207
6       12
7        5
1        1
Name: NameLength, dtype: int64

In [75]:
# Which has one word?
data157[data157['NameLength']==1]

,NAME,STNAME,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,POPESTIMATE_MEAN,NameLength
45333,[Princeton],New Jersey,28592,28086,29388,29714,29661,30115,31002,31368,31270,31187,30038,1


In [76]:
# Create Feature To Determine If Record is County Balance for Easy Filtering.
data157['IsBalance'] = data157['NAME'].apply(lambda x: 1 if x[0] == 'Balance' else 0)

In [77]:
# Copy Last Word to New Column - trailing word distribution.
data157['LastWord'] = data157['NAME'].apply(lambda x: x[-1])
data157['LastWord'].value_counts()

city            9510
town            4115
village         3534
County          2913
(pt.)           2328
borough         1208
Parish            58
Borough           11
Area              10
(balance)          8
government         5
township           5
municipality       4
county             1
corporation        1
CDP                1
Princeton          1
City               1
Name: LastWord, dtype: int64

In [78]:
# Remove Excess Words in City Names: (pt.), city, town, village
data157['NAME'] = data157['NAME'].apply(lambda x: x[:-1] if x[-1] == '(pt.)' else x)
data157['LastWord'] = data157['NAME'].apply(lambda x: x[-1])
data157['NAME'] = data157['NAME'].apply(lambda x: x[:-1] if x[-1] == 'city' or x[-1] == 'town' or x[-1] == 'village' else x)
data157['LastWord'].value_counts()

city            10957
town             4572
village          3936
County           2913
borough          1230
Parish             58
Borough            11
Area               10
(balance)           8
government          5
township            5
municipality        4
county              1
corporation         1
CDP                 1
Princeton           1
City                1
Name: LastWord, dtype: int64

In [79]:
# Now, how long are the names now?
data157['NameLength'] = data157['NAME'].apply(lambda x: len(x))
data157['NameLength'].value_counts()

1    14929
2     5141
4     2849
3      604
5      177
6       12
7        2
Name: NameLength, dtype: int64

In [80]:
# Restore Name to String
data157['NAME'] = data157['NAME'].apply(lambda x: ' '.join(x))
data157.drop(['LastWord'],axis=1, inplace=True)
before = data157.shape[0]

In [81]:
# Aggregate All Parts of Same City
df = data157.groupby(['STNAME','NAME']).agg(sum)
df = df.reset_index()
data157 = df.copy()
after = data157.shape[0]

In [82]:
# Record Reduction
diff = before - after
print('Before: ' + str(before)+'\nAfter: '+str(after)+ '\nDifference: ' + str(diff))

Before: 23714
After: 22475
Difference: 1239


In [83]:
# Minor Name Tweaks
data157['NAME'] = data157['NAME'].replace(['New York'],'New York City')
data157['NAME'] = data157['NAME'].replace(['Nashville-Davidson metro government'],'Nashville')

---
### <b>Objective #3</b>: Calculate YoY Growth.

In [84]:
def get_growth(data, popestimate_i, popestimate_f):
    return ( (data157[popestimate_f] - data157[popestimate_i]) / data157[popestimate_i])

data157['POPGROWTH%2010'] = get_growth(data157,'POPESTIMATE2010', 'POPESTIMATE2011')
data157['POPGROWTH%2011'] = get_growth(data157,'POPESTIMATE2011', 'POPESTIMATE2012')
data157['POPGROWTH%2012'] = get_growth(data157,'POPESTIMATE2012', 'POPESTIMATE2013')
data157['POPGROWTH%2013'] = get_growth(data157,'POPESTIMATE2013', 'POPESTIMATE2014')
data157['POPGROWTH%2014'] = get_growth(data157,'POPESTIMATE2014', 'POPESTIMATE2015')
data157['POPGROWTH%2015'] = get_growth(data157,'POPESTIMATE2015', 'POPESTIMATE2016')
data157['POPGROWTH%2016'] = get_growth(data157,'POPESTIMATE2016', 'POPESTIMATE2017')
data157['POPGROWTH%2017'] = get_growth(data157,'POPESTIMATE2017', 'POPESTIMATE2018')
data157['POPGROWTH%2018'] = get_growth(data157,'POPESTIMATE2018', 'POPESTIMATE2019')

In [85]:
data157.head()

,STNAME,NAME,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,POPESTIMATE_MEAN,NameLength,IsBalance,POPGROWTH%2010,POPGROWTH%2011,POPGROWTH%2012,POPGROWTH%2013,POPGROWTH%2014,POPGROWTH%2015,POPGROWTH%2016,POPGROWTH%2017,POPGROWTH%2018
0,Alabama,Abbeville,2699,2694,2643,2628,2608,2600,2584,2575,2571,2560,2616,1,0,-0.001853,-0.018931,-0.005675,-0.007610,-0.003067,-0.006154,-0.003483,-0.001553,-0.004278
1,Alabama,Adamsville,4500,4493,4471,4449,4420,4390,4356,4327,4308,4281,4400,1,0,-0.001556,-0.004897,-0.004921,-0.006518,-0.006787,-0.007745,-0.006657,-0.004391,-0.006267
2,Alabama,Addison,751,750,743,742,739,734,731,726,723,718,736,1,0,-0.001332,-0.009333,-0.001346,-0.004043,-0.006766,-0.004087,-0.006840,-0.004132,-0.006916
3,Alabama,Akron,355,347,347,343,338,339,333,332,331,328,339,1,0,-0.022535,0.000000,-0.011527,-0.014577,0.002959,-0.017699,-0.003003,-0.003012,-0.009063
4,Alabama,Alabaster,31209,31375,31684,31980,32182,32772,33017,33275,33413,33487,32439,1,0,0.005319,0.009849,0.009342,0.006316,0.018333,0.007476,0.007814,0.004147,0.002215


---
### <b>Objective #4</b>: Transform into OLAP Form

In [86]:
data157.drop(['NameLength', 'POPESTIMATE_MEAN'], axis=1, inplace=True)
data157.head()

,STNAME,NAME,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,IsBalance,POPGROWTH%2010,POPGROWTH%2011,POPGROWTH%2012,POPGROWTH%2013,POPGROWTH%2014,POPGROWTH%2015,POPGROWTH%2016,POPGROWTH%2017,POPGROWTH%2018
0,Alabama,Abbeville,2699,2694,2643,2628,2608,2600,2584,2575,2571,2560,0,-0.001853,-0.018931,-0.005675,-0.007610,-0.003067,-0.006154,-0.003483,-0.001553,-0.004278
1,Alabama,Adamsville,4500,4493,4471,4449,4420,4390,4356,4327,4308,4281,0,-0.001556,-0.004897,-0.004921,-0.006518,-0.006787,-0.007745,-0.006657,-0.004391,-0.006267
2,Alabama,Addison,751,750,743,742,739,734,731,726,723,718,0,-0.001332,-0.009333,-0.001346,-0.004043,-0.006766,-0.004087,-0.006840,-0.004132,-0.006916
3,Alabama,Akron,355,347,347,343,338,339,333,332,331,328,0,-0.022535,0.000000,-0.011527,-0.014577,0.002959,-0.017699,-0.003003,-0.003012,-0.009063
4,Alabama,Alabaster,31209,31375,31684,31980,32182,32772,33017,33275,33413,33487,0,0.005319,0.009849,0.009342,0.006316,0.018333,0.007476,0.007814,0.004147,0.002215


In [87]:
# Extract Dataframe Components
names = data157[['NAME', 'STNAME', 'IsBalance']].copy()
pop_estimates = data157.iloc[:, 2:12]
popgrowth_estimates = data157.iloc[:, 13:23]

# Create 2 Dataframes: Population, Population Growth
population_df = pd.concat([names, pop_estimates], axis=1)
growth_df = pd.concat([names,popgrowth_estimates], axis=1)

population_df.head()

,NAME,STNAME,IsBalance,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019
0,Abbeville,Alabama,0,2699,2694,2643,2628,2608,2600,2584,2575,2571,2560
1,Adamsville,Alabama,0,4500,4493,4471,4449,4420,4390,4356,4327,4308,4281
2,Addison,Alabama,0,751,750,743,742,739,734,731,726,723,718
3,Akron,Alabama,0,355,347,347,343,338,339,333,332,331,328
4,Alabaster,Alabama,0,31209,31375,31684,31980,32182,32772,33017,33275,33413,33487


In [88]:
# Function to transform into Cube
import re
get_year = "\d+"

def getcube(df, FactType):
    state = []
    city = []
    year = []
    balance = []
    fact = []
    for ind in df.index:
        for column in df.columns[3:]:
            state.append(df.loc[ind, 'STNAME'])
            city.append(df.loc[ind, 'NAME']) 
            year.append(
                int(re.findall(get_year, column)[0])
            )
            balance.append(df.loc[ind, 'IsBalance']) 
            fact.append(df.loc[ind, column])
    mydata = {'State':state,'City': city,'Year': year, 'IsBalance': balance, FactType: fact}
    newdataframe = pd.DataFrame(data=mydata)
    return newdataframe
        


In [89]:
# Call Function, Make Cubes, Merge Cubes
population_cube = getcube(population_df, 'Population')
popgrowth_cube = getcube(growth_df, 'Population Growth')
OLAP = pd.merge(population_cube, popgrowth_cube, how='left', on=['State','City','Year', 'IsBalance'])
OLAP.sample(10)

,State,City,Year,IsBalance,Population,Population Growth
147314,North Dakota,Rogers,2014,0,45,-0.022222
140621,North Carolina,High Shoals,2011,0,707,0.000000
83608,Louisiana,Mansura,2018,0,1365,-0.009524
2494,Alabama,Grove Hill,2014,0,1854,-0.007551
116854,Missouri,Vandiver,2014,0,71,0.000000
66598,Iowa,Rossie,2018,0,66,0.030303
214600,West Virginia,Balance of Wyoming County,2010,1,20087,-0.011052
179297,South Carolina,Chapin,2017,0,1588,0.011965
171931,Pennsylvania,Greenville borough,2011,0,5969,-0.008042
30823,Georgia,Claxton,2013,0,2308,-0.002600


In [90]:
#OLAP.to_csv('OLAP_CityPop_sumlev157.csv', index=False)